In [1]:
import datetime
import pytz
import pandas as pd
import numpy as np

In [2]:
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
from ibapi import wrapper
from ibapi.common import *  # Includes many shared datatypes
from ibapi.ticktype import *  # For tick data
from ibapi.contract import Contract
from ibapi.contract import ContractDetails

In [3]:
import logging
import json

In [4]:
# Custom JSON formatter for logs
class JsonFormatter(logging.Formatter):
    def format(self, record):
        # Initialize log record with timestamp and level first
        log_record = {
            "timestamp": self.formatTime(record, self.datefmt),
            "level": record.levelname
        }

        # If the message is a dictionary, merge it into the log record
        if isinstance(record.msg, dict):
            log_record.update(record.msg)
        else:
            # Fallback for non-dictionary messages
            log_record["message"] = record.getMessage()

        # Return the JSON-formatted string with indentation
        return json.dumps(log_record, indent=2)

# Configure the logger
# log_file_path = '/Volumes/LaCie_d2_Professional_Media/ibapi_callbacks/callbacks.log'
log_file_path = '/Users/meow/trading-platform/callbacks.log' #local

file_handler = logging.FileHandler(log_file_path)
file_handler.setLevel(logging.INFO)
file_handler.setFormatter(JsonFormatter())

logger = logging.getLogger("JsonLogger")
logger.setLevel(logging.INFO)

# Ensure no duplicate handlers
if not logger.hasHandlers():
    logger.addHandler(file_handler)

# Example usage for your application logs
trade_app_logger = logging.getLogger('TradeApp')

# Ensure TradeApp logger uses the same handlers
trade_app_logger.setLevel(logging.INFO)
trade_app_logger.addHandler(file_handler)

# Example log
trade_app_logger.info("IBAPI log initialized.")

### Create callback functions here to handle responses from API

In [42]:
class TradeApp(EWrapper, EClient):
    def __init__(self, json_indent=2):
        EWrapper.__init__(self)
        EClient.__init__(self, self)
        self.json_indent = json_indent  # Control JSON indentation
        self.reqId_to_symbol = {}  # Map reqId to symbols for easier tracking

    
    def accountSummary(self, reqId: int, account: str, tag: str, value: str, currency: str):
        print(f"Account Summary Update - Request ID: {reqId}, Account: {account}, Tag: {tag}, Value: {value}, Currency: {currency}")

    def accountSummaryEnd(self, reqId: int):
        print(f"Account Summary End - Request ID: {reqId}")

    def contractDetails(self, reqId: int, contractDetails: ContractDetails):
        print(f"Contract Details for Request ID {reqId}:")
        print(f"Symbol: {contractDetails.contract.symbol}")
        print(f"Exchange: {contractDetails.contract.exchange}")
        print(f"Currency: {contractDetails.contract.currency}")
        print(f"Last Trade Date: {contractDetails.contract.lastTradeDateOrContractMonth}")
        print(f"Multiplier: {contractDetails.contract.multiplier}")
        print(f"Trading Hours: {contractDetails.tradingHours}")
        print(f"Liquid Hours: {contractDetails.liquidHours}")
        print(f"Price Magnifier: {contractDetails.priceMagnifier}")

    def contractDetailsEnd(self, reqId: int):
        """Called when all contract details are received."""
        print(f"Contract Details End - Request ID: {reqId}")        
    
    def log_and_print(self, log_msg):
        """Helper method to log and optionally print JSON-formatted messages."""
        logger.info(log_msg)
        if PrintInLine:
            print(json.dumps(log_msg, indent=self.json_indent))

    def tickPrice(self, reqId: int, tickType: int, price: float, attrib):
        """Capture tick price, including margin-related fields."""
        if tickType == 87:  # Field ID for margin-related tick data
            log_msg = {
                "type": "MarginData",
                "details": {
                    "symbol": self.reqId_to_symbol.get(reqId, "Unknown"),
                    "margin": price,
                    "tickType": tickType
                }
            }
            self.log_and_print(log_msg)

    def tickSize(self, reqId: int, tickType: int, size: int):
        """Capture tick size if needed."""
        log_msg = {
            "type": "TickSize",
            "details": {
                "reqId": reqId,
                "tickType": tickType,
                "size": size
            }
        }
        self.log_and_print(log_msg)

    def tickSnapshotEnd(self, reqId: int):
        """Triggered when snapshot data is complete."""
        log_msg = {
            "type": "TickSnapshotEnd",
            "details": {"reqId": reqId}
        }
        self.log_and_print(log_msg)

### Initiate thread for jupyter kernel communication to API

In [43]:
import threading #Jupyter Notebook Execution: If app.run() blocks the Jupyter kernel, you can use threading

app = TradeApp()
def run_loop():
    app.run()
app.connect("127.0.0.1", 7496, clientId=2)

# Start the API in a separate thread
api_thread = threading.Thread(target=run_loop, daemon=True)
api_thread.start()

if PrintInLine:
    print("IBAPI log initialized to this Jupyter thread.")

ERROR -1 2104 Market data farm connection is OK:usfarm.nj
ERROR -1 2104 Market data farm connection is OK:usfuture
ERROR -1 2104 Market data farm connection is OK:cashfarm
ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefnj


Contract Details for Request ID 9002:
Symbol: CL
Exchange: NYMEX
Currency: USD
Last Trade Date: 20250220
Multiplier: 1000
Trading Hours: 20250113:1800-20250114:1700;20250114:1800-20250115:1700;20250115:1800-20250116:1700;20250116:1800-20250117:1700;20250118:CLOSED;20250119:1800-20250120:1430
Liquid Hours: 20250114:0930-20250114:1700;20250115:0930-20250115:1700;20250116:0930-20250116:1700;20250117:0930-20250117:1700;20250118:CLOSED;20250119:1800-20250120:1430
Price Magnifier: 1
Contract Details End - Request ID: 9002


# IBAPI Account Request Tags
*Requests use the built-in API calls. Be sure to have response handling by defining callback functions in class `TradeApp`*

These tags provide essential account and margin details critical for day trading and algo trading in **equities** and **futures**:

## Core Account Metrics
- **`accountType`**: Type of account (e.g., Cash, Margin).
- **`NetLiquidation`**: Total account value (cash + positions).
- **`TotalCashValue`**: Cash balance, including unrealized futures PnL.
- **`SettledCash`**: Settled cash available (same as `TotalCashValue` for cash accounts).
- **`AccruedCash`**: Net accrued interest.

## Buying Power & Margin
- **`BuyingPower`**: Maximum marginable amount for US stocks.
- **`EquityWithLoanValue`**: Account equity inclusive of loan/margin.
- **`RegTEquity`**: Regulation T equity requirement.
- **`RegTMargin`**: Regulation T margin requirement.
- **`SMA`**: Special Memorandum Account (used for calculating buying power).
- **`InitMarginReq`**: Initial margin requirement for current positions.
- **`MaintMarginReq`**: Maintenance margin requirement.

## Risk Management
- **`AvailableFunds`**: Funds available for trading after margin.
- **`ExcessLiquidity`**: Cushion amount before liquidation risks.
- **`Cushion`**: Percentage of excess liquidity to net liquidation value.
- **`HighestSeverity`**: Severity level of nearing liquidation.

## Look-Ahead Metrics
- **`LookAheadInitMarginReq`**, **`LookAheadMaintMarginReq`**: Predicted margin requirements.
- **`LookAheadAvailableFunds`**, **`LookAheadExcessLiquidity`**: Predicted available funds and liquidity.
- **`LookAheadNextChange`**: Time of next look-ahead margin update.

## Trading-Specific
- **`GrossPositionValue`**: Total absolute value of all positions.
- **`DayTradesRemaining`**: Number of allowed day trades before Pattern Day Trader (PDT) detection (`-1` means unlimited).
- **`Leverage`**: Ratio of `GrossPositionValue` to `NetLiquidation`.

## Ledger (Cash Balances)
- **`$LEDGER`**: All cash balances in base currency.
- **`$LEDGER:CURRENCY`**: Cash balances in a specific currency.
- **`$LEDGER:ALL`**: Cash balances across all currencies.

---

## Why These Matter
- **Risk Management**: Use `ExcessLiquidity`, `Cushion`, and `HighestSeverity` to monitor liquidation risk.
- **Margin Awareness**: Track `BuyingPower`, `InitMarginReq`, and `MaintMarginReq` for margin-sensitive strategies.
- **Day Trading Rules**: Monitor `DayTradesRemaining` to stay compliant with PDT rules.
- **Capital Allocation**: Utilize `TotalCashValue` and `NetLiquidation` for proper strategy allocation.
- **Position Scaling**: Use `Leverage` and `GrossPositionValue` for controlling portfolio size.

---

This information helps ensure you maintain proper account risk management and optimize your trading strategies programmatically.

In [33]:
app.reqAccountSummary(9001, "All", 'TotalCashValue,DayTradesRemaining,BuyingPower,Cushion') # creates a subscription your account data, refreshing periodically and updating in real-time while the subscription is active.

# app.cancelAccountSummary(9001) #closes out the subscription

In [17]:
app.reqAccountUpdates(True, "All")

In [13]:
app.disconnect()

In [44]:
def create_futures_contract(symbol, expiry, exchange="GLOBEX", currency="USD"):
    contract = Contract()
    contract.symbol = symbol        # e.g., "MES"
    contract.secType = "FUT"        # Futures
    contract.exchange = exchange    # "GLOBEX"
    contract.currency = currency    # "USD"
    contract.lastTradeDateOrContractMonth = expiry  # e.g., "202503" for March 2025
    return contract

# Example usage
futures_contract = create_futures_contract("CL", "202503", "NYMEX")


# Request contract details (including margins)
app.reqContractDetails(9002, futures_contract)

In [ ]:
app.reqContractDetails(9003,)

In [40]:
futures_contract

4465945536: 0,MES,FUT,202503,,0,,,GLOBEX,,USD,,,False,,,,combo:

In [60]:
app.reqPositions()

### disconnect from server here

In [18]:
app.disconnect()